In [14]:
pip install transformers==4.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import pandas as pd
import numpy as np
import math 
from sklearn.model_selection import train_test_split
import torch
import torch.nn
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

import logging
logging.basicConfig(level=logging.ERROR)
from torch.nn.utils.rnn import pad_sequence

In [16]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)# to use gpu if possible

cpu


In [17]:
import json
import networkx as nx

# Read the JSONL file and parse the data
def read_fever_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

# Create the knowledge graph
def create_knowledge_graph(data):
    kg = nx.DiGraph()
    
    for item in data:
        claim_id = item['id']
        claim_text = item['claim']
        label = item['label']
        
        # Add claim node
        kg.add_node(claim_id, label="claim", text=claim_text)
        
        if label != "NOT ENOUGH INFO":
            for evidence_group in item['evidence']:
                for evidence in evidence_group:
                    evidence_id = evidence[1]
                    evidence_title = evidence[2]
                    evidence_sentence_num = evidence[3]
                    
                    # Add evidence node
                    kg.add_node(evidence_id, label="evidence", title=evidence_title, sentence_num=evidence_sentence_num)
                    
                    # Add edge between claim and evidence with the relationship label
                    kg.add_edge(claim_id, evidence_id, label=label)
    
    return kg

file_path = "/content/drive/MyDrive/NAM/train.jsonl"
data = read_fever_jsonl(file_path)
knowledge_graph = create_knowledge_graph(data)

In [18]:
print("Nodes in the knowledge graph:", knowledge_graph.number_of_nodes())
print("Edges in the knowledge graph:", knowledge_graph.number_of_edges())

Nodes in the knowledge graph: 268910
Edges in the knowledge graph: 221476


In [19]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
def find_top_k_evidences(kg, claim_id, k=5):
    edges = [(evidence_id, data['label']) for _, evidence_id, data in kg.out_edges(claim_id, data=True)]
    evidences = sorted(edges, key=lambda x: x[1], reverse=True)[:k]
    return evidences

In [22]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def prepare_dataset(kg):
    dataset = []
    for claim_id, data in kg.nodes(data=True):
        if data['label'] == 'claim':
            claim_text = preprocess_text(data['text'])
            evidences = find_top_k_evidences(kg, claim_id)
            for evidence_id, relationship in evidences:
                evidence_data = kg.nodes[evidence_id]
                evidence_text = preprocess_text(evidence_data['title'])
                dataset.append((claim_text, evidence_text, relationship))
    return dataset

dataset = prepare_dataset(knowledge_graph)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [23]:
def extract_features(dataset):
    vectorizer = TfidfVectorizer()
    claims, evidences, labels = zip(*dataset)
    claim_features = vectorizer.fit_transform(claims)
    evidence_features = vectorizer.transform(evidences)
    return claim_features, evidence_features, labels, vectorizer

claim_features, evidence_features, labels, vectorizer = extract_features(dataset)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

X = hstack([claim_features, evidence_features])
y = [1 if label == "SUPPORTS" else 0 for label in labels]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

Accuracy: 0.8110532407407407
Precision: 0.8108741504569955
Recall: 0.9699484189280108
F1 Score: 0.8833065277884148


In [26]:
def predict_claim_veracity(claim, kg, model, vectorizer):
    claim = preprocess_text(claim)
    claim_features = vectorizer.transform([claim])

    predictions = []
    for _, data in kg.nodes(data=True):
        if data['label'] == 'evidence':
            evidence_text = preprocess_text(data['title'])
            evidence_features = vectorizer.transform([evidence_text])
            features = hstack([claim_features, evidence_features])
            pred = model.predict(features)
            prob = model.predict_proba(features)[:, 1][0]
            predictions.append((evidence_text, pred, prob))

    sorted_predictions = sorted(predictions, key=lambda x: x[2], reverse=True)
    top_prediction = sorted_predictions[0]

    return "SUPPORTS" if top_prediction[1] == 1 else "REFUTES", top_prediction[0], top_prediction[2]




In [27]:
def find_claim_id_by_text(kg, claim_text):
    for node_id, data in kg.nodes(data=True):
        if data['label'] == 'claim' and data['text'] == claim_text:
            return node_id
    return None

def predict_claim_veracity(claim_text, kg, model, vectorizer):
    claim_id = find_claim_id_by_text(kg, claim_text)
    print(claim_id)
    if claim_id is None:
        print("Claim not found in the knowledge graph.")
        return None

    claim = preprocess_text(claim_text)
    claim_features = vectorizer.transform([claim])

    top_k_evidences = find_top_k_evidences(kg, claim_id)
    predictions = []
    for evidence_id, relationship in top_k_evidences:
        evidence_data = kg.nodes[evidence_id]
        evidence_text = preprocess_text(evidence_data['title'])
        evidence_features = vectorizer.transform([evidence_text])
        features = hstack([claim_features, evidence_features])
        
        prediction = model.predict(features)
        probability = model.predict_proba(features)

        if prediction[0] == 1:
            predicted_relationship = "SUPPORTS"
        else:
            predicted_relationship = "REFUTES"

        confidence = max(probability[0])

        predictions.append((predicted_relationship, evidence_text, confidence))

    return predictions, top_k_evidences




In [28]:
def find_claim_id_by_text(kg, claim_text):
    for node_id, data in knowledge_graph.nodes(data=True):
        if data['label'] == 'claim' and data['text'] == claim_text:
            return node_id
    return None

def predict_claim_veracity(claim_text, kg, model, vectorizer):
    claim_id = find_claim_id_by_text(knowledge_graph, claim_text)
    if claim_id is None:
        print("Claim not found in the knowledge graph.")
        return None

    claim = preprocess_text(claim_text)
    claim_features = vectorizer.transform([claim])

    top_k_evidences = find_top_k_evidences(kg, claim_id)
    predictions = []
    for evidence_id, relationship in top_k_evidences:
        evidence_data = kg.nodes[evidence_id]
        evidence_text = preprocess_text(evidence_data['title'])
        evidence_features = vectorizer.transform([evidence_text])
        features = hstack([claim_features, evidence_features])
        
        prediction = model.predict(features)
        probability = model.predict_proba(features)

        if prediction[0] == 1:
            predicted_relationship = "SUPPORTS"
        else:
            predicted_relationship = "REFUTES"

        confidence = max(probability[0])

        predictions.append((predicted_relationship, evidence_text, confidence))

    return predictions, top_k_evidences




In [29]:
# Fact Checking function
def fact_check(claim_text):
    predictions, top_k_evidences = predict_claim_veracity(claim_text, knowledge_graph, model, vectorizer)
    print(top_k_evidences)
    if predictions is not None:
      print("Predictions:")
      for relationship, evidence, confidence in predictions:
          print(f"{relationship} with evidence: {evidence} and confidence: {confidence:.2f}")
